# Run the final state algorithm on a ROOT HSDATA file

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.
The string arguements set which particles you would like to use their pre-determined PID for and which you would like to be inclusive (i.e. any number of thse particles).
**Example**, 
1. for tagged photon experiment I might want fs("Beam","Beam")
This means use predetermined photon ID (i.e. I know it came from the tagger) and I will analyse events with any number of tagger photons.
2. fs("e-","proton:pi+:pi-")  i.e. use predetemined electron PID and just determine hadron species based on charge and try all combinations in events with any number of proton,pi+,pi-, but with the number of electrons defined for the topology
3. fs("NONE","ALL") => do not use any PID info other than charge, analyse events with any number of particles (this can be limited by SetMaxParticles)

In [ ]:
gROOT->ProcessLine(".x LoadPi2.C+"); //Load the classes

In [ ]:
 Pi2 fs("NONE","ALL"); //string arguments => PID, INCLUSIVE
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [ ]:
  //strings = treename and filename (give full path)
  auto output = new FiledTree("FinalTree","OutPUT.root");
  fs.FinalStateOutTree(output->Tree()); //connect ouput tree to final state branches

Here we add the cuts for each particle. This is done via a HS::Cuts object which gets propogated to each particle in each topology. To specify any type of cut behaviour you need to generate your own Cuts Method. Here we just use a predefined delta time cut but you could make your won class which has momentum dependence as well.

Here I use a predefined CLAS12DeltaTime cuts class for PID. It takes 4 arguments which are cuts centred on zero for FT, FD, CD, FDCAL (for neutra) respectively.

I define specific cuts for proton and electron, while restrict other particles (here just pions) to only be in CD.

In [ ]:
ParticleCutsManager cutsman;
auto eCut=make_shared<CLAS12DeltaTime>(4,4,0,0); //4ns FT,FD
auto protCut=make_shared<CLAS12DeltaTime>(0,5,5,0); //5ns FD,CD
auto otherCut=make_shared<CLAS12DeltaTime>(0,5,0,0); //5ns FD
cutsman.AddParticleCut("e-",eCut); //assign to manager
cutsman.AddParticleCut("proton",protCut); //assign to manager
cutsman.SetDefaultCut(otherCut); //assign to manager
cutsman.ConfigureCuts(&fs);   

Now register the cuts manager as a post topology function action. This will be applied after the Topo_0 etc functions when start times etc have been determined.

In [ ]:
fs.RegisterPostTopoAction(&cutsman);

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.

**Particle Monitoring** here we are going to make seperate trees for each topology and only keep the detected particle information in each one. The string given to TreePrepManager should be the full path to the output file directory. 
Note as this is registered as a post work action all cuts, including any in Pi2::Kinematics, will be applied.

In [ ]:
auto treeman=new TreePrepManager("particleTrees");
treeman->ConfigureTreeParticles(&fs); //propogate through topologies
fs.RegisterPostWorkAction(treeman); //register post-work i.e. after kinematics is called and all calculations

Now I want to analyse a hipo file so I am going to use the datamanager HipoTrigger which is for full banks files. If I want to analyse just DSTs I should use HipoDST instead of HipoTrigger

In [ ]:
 //create datamanager
  auto dm=std::make_shared<HipoTrigger>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/work/dump/out_clas_pin_004909.evio.00228.hipo");
  dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [ ]:
fs.ProcessData(1E5); //No number given, analyse all events in chain

In [ ]:
%jsroot

Draw some variables from your tree.

In [ ]:
TCanvas c1;
output->Tree()->Draw("MissMass2>>h1(100,-0.5,0.5)","Topo==1");
output->Tree()->Draw("MissMass2","Topo==0","same");
output->Tree()->Draw("MissMass2","Topo==2","same");
output->Tree()->Draw("MissMass2","Topo==3","same");
h1->SetMinimum(0);
c1.Draw();

In [ ]:
TCanvas c3;
c3.Divide(2,2);
c3.cd(1);
treeman->GetPrep(0)->Tree()->Draw("ElectronP:ElectronTime>>h1(100,-10,10,100,0,12)","");
c3.cd(2);
treeman->GetPrep(0)->Tree()->Draw("ProtonP:ProtonTime>>h2(100,-10,10,100,0,10)","","col1");
c3.cd(3);
treeman->GetPrep(0)->Tree()->Draw("PipP:PipTime>>h3(100,-10,10,100,0,10)","","col1");
c3.cd(4);
treeman->GetPrep(0)->Tree()->Draw("PimP:PimTime>>h4(100,-10,10,100,0,10)","","col1");
c3.Draw();

And remember to save the tree!

In [ ]:
delete output;
delete treeman;